# Enable distributed tracing with OpenTelemetry.

When using distributed agents, it is useful to gather information from all the involved components. In this cookbook, we will leverage on using A2A agents as tools as demonstrated in ["Use an Agent as a tool for another agent"](./../a2a_as_tool) to obtain traces from different components and have a consistent view of a distributed trace.

First, a Grafana Tempo instance needs to be running in your environment. A good starting point is the [quick start for tempo](https://grafana.com/docs/tempo/latest/getting-started/docker-example/), removing the `k6-tracing` component.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio. Additionally, we will need some packages handling opentelemetry traces.

In [7]:
%pip install 'any-agent[a2a]'

import nest_asyncio

nest_asyncio.apply()

/Users/jtramon/work/any-agent/449-otel-cookbook/docs/cookbook/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import asyncio
import datetime
import os
from getpass import getpass

import httpx

# This notebook communicates with OpenAI GPT models using the OpenAI API.
if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from opentelemetry.trace import get_tracer_provider

from any_agent import AgentConfig, AgentFramework, AnyAgent
from any_agent.serving import A2AServingConfig
from any_agent.tools import a2a_tool_async

OPENAI_API_KEY not found in environment!
OPENAI_API_KEY set for this session!


/Users/jtramon/work/any-agent/449-otel-cookbook/docs/cookbook/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Add the external exporter

The off-the-shelf OTLP exporter can be used to send the traces generated from the different agents to an observability platform (in our case, Grafana Tempo). We assume that the server is listening locally on port 4318. A helper function will be defined to monitor the availability of the A2A agent.

In [3]:
agent_framework = AgentFramework.OPENAI
agent_model = "gpt-4.1-nano"

# Add another trace exporter
tp = get_tracer_provider()
full_exporter = ConsoleSpanExporter()
http_exporter = OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
tp.add_span_processor(SimpleSpanProcessor(http_exporter))


async def wait_for_server_async(
    server_url: str, max_attempts: int = 20, poll_interval: float = 0.5
):
    """Wait until the server URL is available."""
    attempts = 0

    async with httpx.AsyncClient() as client:
        while True:
            try:
                # Try to make a basic GET request to check if server is responding
                await client.get(server_url, timeout=1.0)
                return  # noqa: TRY300
            except (httpx.RequestError, httpx.TimeoutException):
                # Server not ready yet, continue polling
                pass

            await asyncio.sleep(poll_interval)
            attempts += 1
            if attempts >= max_attempts:
                msg = f"Could not connect to {server_url}. Tried {max_attempts} times with {poll_interval} second interval."
                raise ConnectionError(msg)

## Set up the date agent

An agent that can obtain the current date will now be set up served over A2A. This agent will have a Python function as a tool that will be used in incoming queries.

In [4]:
main_agent = None
served_agent = None
served_task = None
served_server = None

tool_agent_endpoint = "tool_agent"
test_port = 9999


# DATE AGENT


def get_datetime() -> str:
    """Return the current date and time."""
    return str(datetime.datetime.now())


date_agent_description = "Agent that can return the current date."
date_agent_cfg = AgentConfig(
    instructions="Use the available tools to obtain additional information to answer the query.",
    name="date_agent",
    model_id=agent_model,
    description=date_agent_description,
    tools=[get_datetime],
)
date_agent = await AnyAgent.create_async(
    agent_framework=agent_framework,
    agent_config=date_agent_cfg,
)

served_agent = date_agent
(served_task, served_server) = await served_agent.serve_async(
    serving_config=A2AServingConfig(
        port=test_port,
        endpoint=f"/{tool_agent_endpoint}",
        log_level="info",
    )
)
server_url = f"http://localhost:{test_port}/{tool_agent_endpoint}"
await wait_for_server_async(server_url)

INFO:     Started server process [2148]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:9999 (Press CTRL+C to quit)


INFO:     ::1:52116 - "GET /tool_agent HTTP/1.1" 307 Temporary Redirect


INFO:     ::1:52117 - "GET /tool_agent/.well-known/agent.json HTTP/1.1" 200 OK
INFO:     ::1:52122 - "POST /tool_agent HTTP/1.1" 307 Temporary Redirect
INFO:     ::1:52122 - "POST /tool_agent/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2148]


## Set up the main agent

Once the A2A server is up and running and can serve the card for the date agent, the main agent can include that agent as a tool, using the provided functions in the `any-agent` package.

In [5]:
# Date agent is ready for card resolution

main_agent_cfg = AgentConfig(
    instructions="Use the available tools to obtain additional information to answer the query.",
    name="main_agent",
    model_id=agent_model,
    description="The orchestrator that can use other agents via tools using the A2A protocol.",
    tools=[await a2a_tool_async(server_url, http_kwargs={"timeout": 10.0})],
)

main_agent = await AnyAgent.create_async(
    agent_framework=agent_framework,
    agent_config=main_agent_cfg,
)

DATE_PROMPT = (
    "What date and time is it right now? "
    "In your answer please include the year, month, day, and time. "
    "Example answer could be something like 'Today is December 15, 2024'"
)
agent_trace = await main_agent.run_async(DATE_PROMPT)

print(agent_trace.final_output)

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "content": "Use the available tools to obtain additional information to answer the query.",             │ │
│ │     "role": "system"                                                                                        │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What date and time is it right now? In your answer please include the year, month, day, and │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "call_date_agent",                                                                         │ │
│ │     "tool.args": "{\"query\":\"What is the current date and time?\"}"                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 215,                                                                                      │ │
│ │   "output_tokens": 22                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "content": "Use the available tools to obtain additional information to answer the query.",             │ │
│ │     "role": "system"                                                                                        │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What is the current date and time?"                                                         │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_datetime",                                                                            │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 201,                                                                                      │ │
│ │   "output_tokens": 10                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "content": "Use the available tools to obtain additional information to answer the query.",             │ │
│ │     "role": "system"                                                                                        │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "What is the current date and time?"                                                         │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "get_datetime",                                                                            │ │
│ │     "tool.args": "{}"                                                                                       │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 201,                                                                                      │ │
│ │   "output_tokens": 10                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: get_datetime ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ 2025-06-23 10:21:32.326344                                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: get_datetime ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {}                                                                                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ 2025-06-23 10:21:32.326344                                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {"task_status":"completed","data":{"result":"The current date and time is 2025-06-23 10:21:32."}}           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 234,                                                                                      │ │
│ │   "output_tokens": 36                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {"task_status":"completed","data":{"result":"The current date and time is 2025-06-23 10:21:32."}}           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 234,                                                                                      │ │
│ │   "output_tokens": 36                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── EXECUTE_TOOL: call_date_agent ─────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "query": "What is the current date and time?"                                                             │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "contextId": "ffaee063-9ef0-4eed-a52c-80c6d5712b41",                                                      │ │
│ │   "id": "0dfaafdd-0eda-4e2d-993c-867a3ae7e112",                                                             │ │
│ │   "status": {                                                                                               │ │
│ │     "message": {                                                                                            │ │
│ │       "contextId": "ffaee063-9ef0-4eed-a52c-80c6d5712b41",                                                  │ │
│ │       "messageId": "792974da-e2e9-4d1b-a9e9-86a6619d0ac7",                                                  │ │
│ │       "parts": [                                                                                            │ │
│ │         {                                                                                                   │ │
│ │           "text": "{\"result\":\"The current date and time is 2025-06-23 10:21:32.\"}"                      │ │
│ │         }                                                                                                   │ │
│ │       ],                                                                                                    │ │
│ │       "role": "agent",                                                                                      │ │
│ │       "taskId": "0dfaafdd-0eda-4e2d-993c-867a3ae7e112"                                                      │ │
│ │     },                                                                                                      │ │
│ │     "state": "completed",                                                                                   │ │
│ │     "timestamp": "2025-06-23T08:21:33.281199+00:00"                                                         │ │
│ │   }                                                                                                         │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── CALL_LLM: gpt-4.1-nano ─────────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Today is June 23, 2025, and the current time is 10:21:32.                                                   │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 447,                                                                                      │ │
│ │   "output_tokens": 23                                                                                       │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Today is June 23, 2025, and the current time is 10:21:32.


## Observing the traces and stopping the server

Once this example is run, go to the "Explore" section in the left-hand sidebar of the Grafana Tempo browser. Click on the "Run query" button with the default settings. Two traces should appear in the "Table - Traces" section. Note that context propagation does not yet work, so the two traces will have different `trace_id`s. With trace propagation, the `trace_id` of the current trace is sent in an HTTP header and is recreated at the other end. Note that the `A2AClient` and `A2AServer` packages will also include their own traces.

After checking out the traces in the browser, execute the next cell to shut down the A2A server.

In [6]:
if served_server:
    served_server.should_exit = True
    await served_task